In [1]:
import numpy as np
import pandas as pd
import sklearn

In [2]:
raw = pd.read_csv('preprocessed_data.csv')
print(raw['Absenteeism Time in Hours'].median())

3.0


In [3]:
targets = np.where(raw['Absenteeism Time in Hours'] > raw['Absenteeism Time in Hours'].median(), 1,0)
raw['Targets'] = targets
raw = raw.drop(['Absenteeism Time in Hours'], axis = 1)

In [4]:
raw['Targets'].sum() / raw['Targets'].shape[0]
raw_inputs = raw.iloc[:, :19]
raw_targets = raw.iloc[:, -1:]

In [5]:
from sklearn.preprocessing import StandardScaler
raw_inputs.columns.values

array(['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'reason1', 'reason2',
       'reason3', 'age1', 'age2', 'age3', 'Transportation Expense',
       'Distance to Work', 'Daily Work Load Average', 'Body Mass Index',
       'Education', 'Children', 'Pets'], dtype=object)

In [6]:
scaler = StandardScaler()
to_scale = raw_inputs.drop(['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'reason1', 'reason2',
       'reason3', 'age1', 'age2', 'age3','Education'], axis=1)
no_scale = raw_inputs.drop(['Transportation Expense','Distance to Work',
                            'Daily Work Load Average', 'Body Mass Index','Children', 'Pets'], axis = 1)
scaler.fit(to_scale)
to_scale_inputs = scaler.transform(to_scale)
scaled_targets = raw_targets

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [7]:
no_scaled = no_scale.values
scaled_inputs = np.concatenate([to_scale_inputs, no_scaled], axis = 1)
scaled_inputs[0]

array([ 1.00584437,  0.41281584, -0.80633129,  0.76743118,  0.88046927,
        0.26848661,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  0.        ])

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, scaled_targets, train_size = 0.8, shuffle=True, random_state=20)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [10]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(560, 19) (560, 1)
(140, 19) (140, 1)


In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
reg = LinearRegression()

In [13]:
reg.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [14]:
reg.score(x_train, y_train)

0.3021822042697334

In [15]:
pred = reg.predict(x_train)

In [16]:
np.sum(pred) / x_train.shape[0]

0.4517857142857143

In [17]:
weights = reg.coef_

In [18]:
bias = reg.intercept_

In [19]:
raw_inputs.columns.values

array(['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'reason1', 'reason2',
       'reason3', 'age1', 'age2', 'age3', 'Transportation Expense',
       'Distance to Work', 'Daily Work Load Average', 'Body Mass Index',
       'Education', 'Children', 'Pets'], dtype=object)

In [20]:
weights = weights.transpose()

In [21]:
input_columns = raw_inputs.columns.values
sum_table = pd.DataFrame(columns=['features'], data=input_columns)

In [22]:
sum_table['values'] = weights

In [23]:
sum_table.index = sum_table.index + 1
sum_table.loc[0] = ['Bias', reg.intercept_[0]]
sum_table.sort_index(ascending=False)
sum_table

,features,values
1,mon,0.127071
2,tue,-0.015185
3,wed,-0.012297
4,thu,0.040945
5,fri,0.054304
6,sat,-0.043084
7,reason1,-0.023341
8,reason2,-0.057503
9,reason3,-0.037984
10,age1,0.003481


In [25]:
sum_table['odds_ratio'] = np.exp(sum_table['values'])
sum_table.sort_values(['odds_ratio'], ascending=False)

,features,values,odds_ratio
0,Bias,0.770752,2.161391
1,mon,0.127071,1.135498
11,age2,0.126087,1.134381
14,Distance to Work,0.079764,1.083031
5,fri,0.054304,1.055806
16,Body Mass Index,0.045473,1.046523
4,thu,0.040945,1.041794
10,age1,0.003481,1.003487
3,wed,-0.012297,0.987778
2,tue,-0.015185,0.984929


In [26]:
reg.score(x_test, y_test)

0.24680848669561417

In [29]:
import pickle
with open('mymodel', 'wb') as file:
    pickle.dump(reg, file)
with open('myscaler', 'wb') as file:
    pickle.dump(scaler, file)